In [ ]:
resume = "./resumes/cw-resume-jv1.pdf"

**Using pdfminer.six**

- Probably will use. Best so far

In [13]:
from pdfminer.high_level import extract_text, extract_pages

pages = [x for x in extract_pages(resume)]

for page in range(len(pages)):
    print('page_number', page)
    text = extract_text(resume, page_numbers=page)
    print(text)


page_number 0
Chris West

(cid:131) 904-469-4104 # christinewwwest@gmail.com (cid:239) linkedin § github

Technical Skills

Working knowledge: Python, JS/TS, React, HTML/CSS, Tailwind, SQL, Git/GitHub, Linux, Firebase, Contentful CMS,
Jira/Kanban
Exposure: Ruby, C#, C, LaTex

Experience

McMaster University
Web Developer, Contract

Mar – Apr 2023
Hamilton, ON
• Developed and maintained a research facility website using React.js, SCSS, and Contentful CMS, utilizing Git and GitHub

for version control and Jira with a Kanban board for collaboration.

• Ensured website accessibility compliance with WCAG 2.1 guidelines, optimized website performance, and improved website

visibility through SEO techniques.

• Successfully launched the website on time and within budget, after conducting thorough testing for accessibility, SEO, and

performance, resulting in positive feedback from users and stakeholders.

CoLab
Software Developer, Intern

Oct 2022 – Jan 2023
GTA, ON
• Collaborated on an inter

**Using pypdf package**

- Didn't work on Jek's email. Some of the spaces are not being extracted.

In [ ]:
from pypdf import PdfReader

reader = PdfReader(resume)
number_of_pages = len(reader.pages)

for page_number in range(number_of_pages):
    page = reader.pages[page_number]
    text = page.extract_text()
    print(text)

**Using pyPDF2**

- Didn't work on Jek's email. Some of the spaces are not being extracted.

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader(resume, strict=True)
number_of_pages = len(reader.pages)
for page_number in range(number_of_pages):
    page = reader.pages[page_number]
    text = page.extract_text()
    print(text)
